# Narrative Generation Demo

This notebook demonstrates how to use the narrative generation feature with `factual_explanation.plot(style='narrative', ...)`

In [ ]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from calibrated_explanations import CalibratedExplainer

## 1. Prepare Data and Train Model

In [ ]:
# Load iris dataset (binary classification)
X, y = load_iris(return_X_y=True)
mask = y < 2  # Only use classes 0 and 1
X, y = X[mask], y[mask]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Train model
model = RandomForestClassifier(n_estimators=10, random_state=42)
model.fit(X_train, y_train)

print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

## 2. Create Calibrated Explainer

In [ ]:
# Create explainer with feature names
explainer = CalibratedExplainer(
    model, 
    X_train, 
    y_train,
    feature_names=["sepal_length", "sepal_width", "petal_length", "petal_width"]
)

print("Explainer created successfully!")

## 3. Generate Factual Explanations

In [ ]:
# Generate factual explanations for first 5 test instances
factual_explanation = explainer.explain_factual(X_test[:5])

print(f"Generated explanations for {len(factual_explanation.explanations)} instances")

## 4. Generate Narratives - DataFrame Output

**This is the main feature!** Generate human-readable narratives for all expertise levels:

In [ ]:
# Generate narratives as DataFrame
result = factual_explanation.plot(
    style="narrative",
    expertise_level=("beginner", "intermediate", "advanced"),
    output="dataframe"
)

print("Result type:", type(result))
print("\nColumns:", result.columns.tolist())
print("\nDataFrame shape:", result.shape)

# Display the DataFrame
result

## 5. View Individual Narratives

In [ ]:
# View beginner-level narrative for first instance
print("Beginner Narrative for Instance 0:")
print("=" * 80)
print(result.iloc[0]['factual_explanation_beginner'])

In [ ]:
# View advanced-level narrative for first instance
print("Advanced Narrative for Instance 0:")
print("=" * 80)
print(result.iloc[0]['factual_explanation_advanced'])

## 6. Alternative Output Formats

In [ ]:
# Text output - formatted for console/reports
text_result = factual_explanation.plot(
    style="narrative",
    expertise_level="intermediate",
    output="text"
)

print(text_result)

In [ ]:
# Dict output - for programmatic access
dict_result = factual_explanation.plot(
    style="narrative",
    expertise_level="beginner",
    output="dict"
)

print("Dict output - first instance:")
print(dict_result[0])

In [ ]:
# HTML output - for web display
from IPython.display import HTML

html_result = factual_explanation.plot(
    style="narrative",
    expertise_level="beginner",
    output="html"
)

HTML(html_result)

## 7. Single Expertise Level

In [ ]:
# Generate only beginner-level narratives
beginner_only = factual_explanation.plot(
    style="narrative",
    expertise_level="beginner",  # Single level as string
    output="dataframe"
)

print("Columns:", beginner_only.columns.tolist())
beginner_only

## Summary

**Key Points:**
- Use `factual_explanation.plot(style="narrative", ...)` to generate narratives
- Supports 3 expertise levels: `"beginner"`, `"intermediate"`, `"advanced"`
- Supports 4 output formats: `"dataframe"`, `"text"`, `"html"`, `"dict"`
- Works with factual, alternative, and fast explanations
- Automatically detects problem type (classification, regression, etc.)